In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In this notebook, I am collecting everything one needs to run densepose within the google colab environment.

First, we'll mount our google drive

Next, we'll install a 2018 Anaconda2 to then use it to install an old pytorch 1.0.1 -- not the nightly build -- including caffe2, install the COCO API and finally densepose. For the latter, we have to make sure we are using the correct & latest "env.py" from the detectron utils. Finally, we'll fetch the densepose data.

In [0]:
import os
from google.colab import files

!wget -c https://repo.continuum.io/archive/Anaconda2-5.3.0-Linux-x86_64.sh
!chmod +x Anaconda2-5.3.0-Linux-x86_64.sh
!bash ./Anaconda2-5.3.0-Linux-x86_64.sh -b -f -p /usr/local
#!wget -c https://repo.continuum.io/archive/Anaconda2-2019.03-Linux-x86_64.sh
#!chmod +x Anaconda2-2019.03-Linux-x86_64.sh
#!bash ./Anaconda2-2019.03-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python2.7/site-packages/')

#!conda install -y pytorch-nightly -c pytorch
!conda install -y pytorch
!pip install argparse
!pip install pyhamcrest
!pip install protobuf
!pip install future
!pip install pydot
!pip install cython scikit-image easydict

!python2 -c 'from caffe2.python import workspace; print(workspace.NumCudaDevices())'
!python2 -c 'from caffe2.python import core' 2>/dev/null && echo "Success" || echo "Failure"

COCOAPI='/usr/local/cocoapi/'
!git clone https://github.com/cocodataset/cocoapi.git $COCOAPI
  
os.chdir('/usr/local/cocoapi/PythonAPI/')
!python2 setup.py install --user

DENSEPOSE='/usr/local/densepose/'
!git clone https://github.com/facebookresearch/densepose $DENSEPOSE
os.chdir('/usr/local/densepose/detectron/utils/')
!rm env.py
!wget https://raw.githubusercontent.com/facebookresearch/Detectron/master/detectron/utils/env.py
!pip install -r $DENSEPOSE/requirements.txt
os.chdir('/usr/local/densepose/')
!make
!python $DENSEPOSE/detectron/tests/test_spatial_narrow_as_op.py

os.chdir('/usr/local/densepose/DensePoseData/')
!bash get_densepose_uv.sh
!bash get_eval_data.sh

Now we're ready to infer the positions of bodies in images, copy the results into our gdrive and zip them and clear the densepose output directory (so we could run the next batch straight away):

In [0]:
os.chdir('/usr/local/densepose/')
!python2 tools/infer_simple.py \
    --cfg configs/DensePose_ResNet101_FPN_s1x-e2e.yaml \
    --output-dir DensePoseData/infer_out/ \
    --image-ext jpg \
    --wts https://dl.fbaipublicfiles.com/densepose/DensePose_ResNet101_FPN_s1x-e2e.pkl \
    /content/gdrive/My\ Drive/skopdaube/batch5/
  
!cp /usr/local/densepose/DensePoseData/infer_out/*IUV.png /content/gdrive/My\ Drive/skopdaube/batch5Out/
!cp /usr/local/densepose/DensePoseData/infer_out/*INDS.png /content/gdrive/My\ Drive/skopdaube/batch5Out/
!zip -r /content/gdrive/My\ Drive/skopdaube/batch5Out.zip /content/gdrive/My\ Drive/skopdaube/batch5Out/

!rm -r /usr/local/densepose/DensePoseData/infer_out/*

In [0]:
os.chdir('/usr/local/densepose/')
!python2 tools/infer_simple.py \
    --cfg configs/DensePoseKeyPointsMask_ResNet50_FPN_s1x-e2e.yaml \
    --output-dir /content/gdrive/My\ Drive/DensePose/skopdaube/test/ \
    --image-ext jpg \
    --wts https://dl.fbaipublicfiles.com/detectron/ImageNetPretrained/MSRA/R-50.pkl \
    /content/gdrive/My\ Drive/skopdaube/test/